In [1]:
import pandas as pd
import json
import requests
from tqdm import *
import time

headers = {
    'Accept':'application/json, text/plain, */*',
    'Accept-Encoding':'gzip, deflate, br, zstd',
    'Accept-Language':'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    # If-None-Match:"06c0f14de3c0229b8ad093f25a17f6572"
    # Origin:https://www.uefa.com
    # Priority:u=1, i
    # Referer:https://www.uefa.com/
    # Sec-Ch-Ua:"Not/A)Brand";v="8", "Chromium";v="126", "Microsoft Edge";v="126"
    # Sec-Ch-Ua-Mobile:?0
    # Sec-Ch-Ua-Platform:"Windows"
    # Sec-Fetch-Dest:empty
    # Sec-Fetch-Mode:cors
    # Sec-Fetch-Site:same-site
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0'
    # X-Api-Key:ceeee1a5bb209502c6c438abd8f30aef179ce669bb9288f2d1cf2fa276de03f4
}

In [ ]:
def get_matches_info():
    url = "https://match.uefa.com/v5/matches?competitionId=3&fromDate=2024-06-15&limit=55&offset=0&order=ASC&phase=ALL&seasonYear=2024&toDate=2024-07-15&utcOffset=8"
    
    res = requests.get(url,headers=headers)

    data = json.loads(res.text)
    
    return [i['id'] for i in data]

match_ids = get_matches_info()
match_ids

In [19]:
len(match_ids)

51

In [17]:
data

[{'awayTeam': {'associationId': '200117',
   'associationLogoUrl': 'https://img.uefa.com/imgml/uefacom/elements/logos/ma/SCO.svg',
   'bigLogoUrl': 'https://img.uefa.com/imgml/flags/700x700/SCO.png',
   'confederationType': 'UEFA',
   'countryCode': 'SCO',
   'id': '117',
   'idProvider': 'FAME',
   'internationalName': 'Scotland',
   'isPlaceHolder': False,
   'logoUrl': 'https://img.uefa.com/imgml/flags/70x70/SCO.png',
   'mediumLogoUrl': 'https://img.uefa.com/imgml/flags/240x240/SCO.png',
   'organizationId': '200117',
   'teamCode': 'SCO',
   'teamTypeDetail': 'NATIONAL_MEN_TEAM_A',
   'translations': {'countryName': {'EN': 'Scotland',
     'FR': 'Écosse',
     'DE': 'Schottland',
     'ES': 'Escocia',
     'PT': 'Escócia',
     'IT': 'Scozia',
     'RU': 'Шотландия',
     'ZH': '苏格兰',
     'TR': 'Scotland',
     'HU': 'Scotland',
     'CS': 'Scotland',
     'NL': 'Scotland'},
    'displayName': {'EN': 'Scotland',
     'FR': 'Écosse',
     'DE': 'Schottland',
     'ES': 'Escocia',


In [22]:
def get_match_data(id):
    url = "https://match.uefa.com/v5/matches?matchId=%s" % id
    res = requests.get(url,headers=headers)
    data = json.loads(res.text)[0]
    #   'id', 比赛id
    #   'awayTeam', 客场球队信息
    #       'id' 客场球队id
    #       'translation'
    #           'countryname'
    #               'ZH' 客场球队中文名称
    #   'homeTeam', 主场球队信息
    #       'id' 主场球队id
    #       'translation'
    #           'countryname'
    #               'ZH' 主场球队中文名称
    #   'score', 比分
    #       'regular' 常规时间比分
    #           'away' 客场球队得分
    #           'home' 主场球队得分
    #       'total' 全场比赛比分
    #           'away' 客场球队得分
    #           'home' 主场球队得分
    #   'playerOfTheMatch', 本场mvp球员
    #       'player'
    #           'translation'
    #               'name'
    #                   'ZH' 球员中文名
    #               'nationalFieldPosition'
    #                   'ZH' 球员位置（此项目中使用国家队位置）
    #   'playerEvents', 球员事件，红黄牌、进球等

    #   'behindClosedDoors', 非公开?不重要
    #   'competition', 比赛基本信息，比如成人组、男子、比赛类别（杯赛）等
    #   'fullTimeAt', 完赛时间
    #   'group', 组
    #   'kickOffTime', 比赛开始时间
    #   'lineupStatus',
    #   'matchAttendance',
    #   'matchNumber',
    #   'matchday', 比赛日信息
    #   'referees', 裁判信息
    #   'round', 系列赛阶段信息，小组赛、淘汰赛等
    #   'seasonYear', 
    #   'sessionNumber',
    #   'stadium', 球馆信息
    #   'status', 比赛状态
    #   'type', 小组赛？这个字段不确定
    #   'winner' 获胜球队

    need_match_data = [
        data['id'],
        data['awayTeam']['id'],
        data['awayTeam']['translations']['countryName']['ZH'],
        data['homeTeam']['id'],
        data['homeTeam']['translations']['countryName']['ZH'],
        data['score']['regular']['away'],
        data['score']['regular']['home'],
        data['score']['total']['away'],
        data['score']['total']['home'],
        data['playerOfTheMatch']['player']['translations']['name']['ZH'],
        data['playerOfTheMatch']['player']['translations']['nationalFieldPosition']['ZH'],
    ]
    return need_match_data

# match_data = get_match_data(2036161)
match_data = []
for i in tqdm(match_ids):
    match_data.append(get_match_data(i))
    time.sleep(0.5)
    
match_data = pd.DataFrame(match_data,columns=['match_id','away_team_id','away_team_name','home_team_id','home_team_name','away_score','home_score','total_away_score','total_home_score','mvp_name','mvp_position'])
match_data.to_excel('..\\数据\\match_data.xlsx',index=False)


In [21]:
match_data

['2036161', '117', '苏格兰', '47', '德国', 1, 5, 1, 5, '贾马尔·穆夏拉', '中场球员']

In [5]:
url = "https://match.uefa.com/v5/matches?matchId=%d" % 2036161
res = requests.get(url,headers=headers)
data = json.loads(res.text)
data

[{'awayTeam': {'associationId': '200117',
   'associationLogoUrl': 'https://img.uefa.com/imgml/uefacom/elements/logos/ma/SCO.svg',
   'bigLogoUrl': 'https://img.uefa.com/imgml/flags/700x700/SCO.png',
   'confederationType': 'UEFA',
   'countryCode': 'SCO',
   'id': '117',
   'idProvider': 'FAME',
   'internationalName': 'Scotland',
   'isPlaceHolder': False,
   'logoUrl': 'https://img.uefa.com/imgml/flags/70x70/SCO.png',
   'mediumLogoUrl': 'https://img.uefa.com/imgml/flags/240x240/SCO.png',
   'organizationId': '200117',
   'teamCode': 'SCO',
   'teamTypeDetail': 'NATIONAL_MEN_TEAM_A',
   'translations': {'countryName': {'EN': 'Scotland',
     'FR': 'Écosse',
     'DE': 'Schottland',
     'ES': 'Escocia',
     'PT': 'Escócia',
     'IT': 'Scozia',
     'RU': 'Шотландия',
     'ZH': '苏格兰',
     'TR': 'Scotland',
     'HU': 'Scotland',
     'CS': 'Scotland',
     'NL': 'Scotland'},
    'displayName': {'EN': 'Scotland',
     'FR': 'Écosse',
     'DE': 'Schottland',
     'ES': 'Escocia',


In [4]:
match_ids

['2036161',
 '2036162',
 '2036163',
 '2036164',
 '2036167',
 '2036165',
 '2036166',
 '2036170',
 '2036169',
 '2036168',
 '2036171',
 '2036172',
 '2036176',
 '2036173',
 '2036174',
 '2036177',
 '2036178',
 '2036175',
 '2036182',
 '2036179',
 '2036180',
 '2036184',
 '2036183',
 '2036181',
 '2036185',
 '2036186',
 '2036187',
 '2036188',
 '2036192',
 '2036191',
 '2036189',
 '2036190',
 '2036194',
 '2036193',
 '2036196',
 '2036195',
 '2036198',
 '2036197']